In [6]:
import pandas as pd
import numpy as np
import os

In [546]:
filepath = './data/'
trainfile = 'train.dat'
actorfile = 'movie_actors.dat'
directorfile =  'movie_directors.dat'
genrafile = 'movie_genres.dat'
movie_tagfile = 'movie_tags.dat'
tagfile =  'tags.dat'
testfile = 'test.dat'
user_taggedfile = 'user_taggedmovies.dat'

In [547]:
import codecs
def read_data(filepath, filename):
    data = []
    with codecs.open(os.path.join(filepath,filename),'r',encoding = "ISO-8859-1") as ff:
        for line in ff:
            line_data = line.split('\t')
            line_data[-1]=line_data[-1][:-2]
            data.append(line_data)
    return data

df_train = pd.read_csv(os.path.join(filepath,trainfile), sep=" ",engine='python')
df_test = pd.read_csv(os.path.join(filepath,testfile), sep=" ",engine='python')
df_actor = pd.DataFrame.from_records(read_data(filepath, actorfile)[1:],columns=read_data(filepath, actorfile)[0])
df_director = pd.DataFrame.from_records(read_data(filepath, directorfile)[1:],
                                        columns=read_data(filepath, directorfile)[0])
df_genre = pd.read_csv(os.path.join(filepath,genrafile), sep="\t",engine='python')
df_movie_tag = pd.read_csv(os.path.join(filepath,movie_tagfile), sep="\t",engine='python')
df_tag = pd.read_csv(os.path.join(filepath,tagfile), sep="\t",engine='python')
df_test = pd.read_csv(os.path.join(filepath,testfile), sep=" ",engine='python')
df_user_tag = pd.read_csv(os.path.join(filepath,user_taggedfile), sep=" ",engine='python')

### These are garbage...Works start from next section

In [417]:
import networkx as nx
def lebel_preprocess(df,primary,attr,i,att_tag=True):
    if type(attr)!=str:
        edge_att = attr[1]
        attr=attr[0]
        att_tag =False
    df[attr] = df[attr].astype('category').cat.codes
    df[attr] = i+df[attr].astype('str')
    df[primary] = primary[0]+df[primary].astype('str')
    if att_tag==True:
        subset = df[[primary, attr]]
        attr_t = [tuple(x) for x in subset.to_numpy()]
    else:
        subset = df[[primary, attr,edge_att]]
        attr_t = [(x[0],x[1],{'w':x[2]}) for x in subset.to_numpy()]
    g=nx.DiGraph()
    g.add_edges_from(attr_t)
    return subset,g


genre_t,g_genre = lebel_preprocess(df_genre.copy(),'movieID' ,'genre', 'g')
director_t, g_dir = lebel_preprocess(df_director.copy(),'movieID', 'directorID', 'd')
actor_t,g_actor = lebel_preprocess(df_actor.copy(),'movieID', ['actorID','ranking'], 'a')
mtag_t,g_mtag = lebel_preprocess(df_movie_tag.copy(), 'movieID',['tagID','tagWeight'], 't')
utag_t,g_utag = lebel_preprocess(df_user_tag.copy(), 'userID','tagID', 't')
utag_movie, g_utag_m = lebel_preprocess(df_user_tag.copy(), 'tagID','movieID', 'm')
u_movie, g_u_m = lebel_preprocess(df_user_tag.copy(), 'userID','movieID', 'm')
um_train, g_train = lebel_preprocess(df_train.copy(), 'userID','movieID', 'm')

In [471]:
utm_g =nx.Graph()
utm_g.add_edges_from(g_utag.edges)
utm_g.add_edges_from(g_utag_m.edges)
um_pa = pd.DataFrame(nx.preferential_attachment(utm_g,g_u_m.edges()))
um_pa[3] =um_pa[0]+um_pa[1]
um_pa_d = dict(zip(um_pa[3],um_pa[2]))

um_jc = pd.DataFrame(nx.jaccard_coefficient(utm_g,g_u_m.edges()))
um_jc[3] =um_jc[0]+um_jc[1]
um_jc_d = dict(zip(um_jc[3],um_jc[2]))

In [473]:
def movie_centrality(df,primary,attr):
    dft = pd.get_dummies(df.set_index(primary)[attr].astype(str)).max(level=0).sort_index()
    s,t = document_centrality(dft.values, 20)
    m_c = dict(zip(dft.index,t))
    return m_c

mg_cent = movie_centrality(genre_t.copy(),'movieID', 'genre')
ma_cent = movie_centrality(actor_t.loc[actor_t['ranking']=='1'].copy(),'movieID', 'actorID')
mt_cent = movie_centrality(mtag_t.copy(),'movieID', 'tagID')

In [475]:
um_train[2]=um_train['userID']+um_train['movieID']
um_train['pa']= um_train.apply(lambda row: um_pa_d[row[2]] if row[2] in um_pa_d else 0,axis=1)
um_train['jc']= um_train.apply(lambda row: um_jc_d[row[2]] if row[2] in um_jc_d else 0,axis=1)

In [497]:
um_train['mg']= um_train.apply(lambda row: mg_cent[row['movieID']] if row['movieID'] in mg_cent
                               else 0,axis=1)
um_train['ma']= um_train.apply(lambda row: ma_cent[row['movieID']] if row['movieID'] in ma_cent
                               else 0,axis=1)
um_train['mt']= um_train.apply(lambda row: mt_cent[row['movieID']] if row['movieID'] in mt_cent
                               else 0,axis=1)
um_train['rating'] = df_train['rating']

In [229]:
import numpy as np


def document_centrality(td, *args):
    """
    calculate recursive term centrality
    :param td: adjacency metrix for bi-partite graph (eg. author-keyword)
    :param args: (eg. no of iterations)
    :return: calculated centrality arrays for the two attributes (eg. author and keyword)
    """
    if len(args) > 0:
        n_it = args[0]
    else:
        n_it = 20
    if n_it < 2:
        n_it = 20
    td = np.array(td)
    n, t = np.shape(td)
    if n > 1 and t > 1:
        l = np.zeros((n, n_it + 1))
        k = np.zeros((t, n_it + 1))
        k0 = td.sum(axis=0)
        l0 = td.sum(axis=1)
        k[:, 0] = k0
        l[:, 0] = l0

        for i in range(1, n_it + 1):
            k[:, i] = (1 / k0).transpose() * td.transpose().dot(l[:, i - 1])
            l[:, i] = (1 / l0) * td.dot(k[:, i - 1])
        x1 = k[:, 0]
        x1[x1 == 0] = np.nan
        x1 = np.log(x1)
        x1 = (x1 - np.nanmean(x1)) / np.nanstd(x1, ddof=1)
        x2 = k[:, 20]
        x2[x2 == 0] = np.nan
        x2 = (x2 - np.nanmean(x2)) / np.nanstd(x2, ddof=1)
        cd = x1 + x2
        x1 = l[:, 0]
        x1[x1 == 0] = np.nan
        x1 = np.log(x1)
        x1 = (x1 - np.nanmean(x1)) / np.nanstd(x1, ddof=1)
        x2 = l[:, 20]
        x2[x2 == 0] = np.nan
        x2 = (x2 - np.nanmean(x2)) / np.nanstd(x2, ddof=1)
        ct = x1 + x2
    else:
        print('Size of the input matrix is too small to do something useful.')
        ct = []
        cd = []
    return cd, ct


# if __name__ == '__main__':
#     td = [[1, 2, 3, 4], [2, 3, 4, 5], [1, 2, 7, 8], [3, 5, 1, 2]]
#     td = [[1, 2, 3], [4, 5, 6], [3, 5, 7], [7, 1, 2], [3, 6, 4], [3, 5, 5]]
#     Cd, Ct = document_centrality(td, 20)
#     print(Cd,Ct)


### Work starts from here

In [606]:
def create_matrix(df_data):
    r_df = df_data.pivot(index = 'userID', columns ='movieID', values = 'rating').fillna(0)
    users = list(r_df.index)
    users_index = {users[i]: i for i in range(len(users))}
    movies = list(r_df.columns)
    movies_index = {movies[i]: i for i in range(len(movies))}
    return r_df, users_index, movies_index

In [607]:
from scipy.sparse.linalg import svds
from scipy.linalg import sqrtm

def svd(train, k):
    r_mat = train.values
    user_ratings_mean = np.mean(r_mat, axis = 1)
    r_norm = r_mat - user_ratings_mean.reshape(-1, 1)
    
    U, s, V = svds(r_norm, k)
    s = np.diag(s)

    # we take only the k most significant features
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]
    s_root=sqrtm(s)
    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)
    UsV = UsV + user_ratings_mean.reshape(-1, 1)
    print("svd done")
    return UsV

In [608]:
def rmse(true, pred):
    # this will be used towards the end
    x = true - pred
    return sum([xi*xi for xi in x])/len(x)

In [620]:
def save_output(items, item):
    if item =="output":
        output = open(os.path.join(filepath,'output.dat'), 'w')
        output.writelines( "%.1f\n" % item for item in items )
        output.close()

In [614]:
df_data = pd.concat([df_train,df_test])
# to test the performance over a different number of features

no_of_features = [50]
utilMat, users_index, items_index = create_matrix(df_data.copy())
for f in no_of_features: 
    svdout = svd(utilMat, k=f)
    pred = [] #to store the predicted ratings
    for _,row in df_test.iterrows():
        user = row['userID']
        item = row['movieID']
        u_index = users_index[user]
        if item in items_index:
            i_index = items_index[item]
            pred_rating = svdout[u_index, i_index]
        else:
            pred_rating = np.mean(svdout[u_index, :])
        pred.append(pred_rating)

# print(rmse(df_test['rating'], pred))

svd done


In [621]:
save_output(pred, 'output')

In [612]:
len(pred)

71299

In [613]:
df_test.shape

(71299, 2)